<a href="https://colab.research.google.com/github/BYU-Hydroinformatics/gwbf-notebooks/blob/main/2_GSLB_PCHIP_DeltaWTE_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Load ML model BFD results

In [1]:
import pandas as pd
import geopandas as gpd
import glob
import os
import time

def load_streamflow_data(data_folder):
    """
    Load all CSV files from the specified folder, parsing 'Date' column as datetime

    Parameters:
    data_folder: Path to the folder containing streamflow CSV files

    Returns:
    Dictionary: Keys are file names (without .csv), values are DataFrames
    """
    streamflow_data = {}
    csv_files = glob.glob(os.path.join(data_folder, '*.csv'))

    for file_path in csv_files:
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        # Parse 'Date' column as datetime while reading the CSV
        df = pd.read_csv(file_path, parse_dates=['Date'])
        streamflow_data[file_name] = df

    print(f"Loaded {len(streamflow_data)} CSV files: {list(streamflow_data.keys())}")
    return streamflow_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/My Drive/xueyi_research/GSLB_gages/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
streamflow_data = load_streamflow_data(data_folder)
measurements = pd.read_csv('GSLB_1900-2023_TS_with_aquifers.csv')

Loaded 68 CSV files: ['10141000', '10026500', '10041000', '10134500', '10157500', '10015900', '10092700', '10105900', '10132000', '10106000', '10128500', '10159500', '10167000', '10133650', '10155500', '10129900', '10118000', '10136500', '10020300', '10171000', '10143500', '10132500', '10155000', '10079500', '10104700', '10153100', '10163000', '10155200', '10038000', '10133800', '10131000', '10168000', '10011500', '10130500', '10126000', '10020100', '10172860', '10152000', '10104900', '10039500', '10046500', '10142000', '10125500', '10109000', '10129300', '10172700', '10139300', '10023000', '10109001', '10113500', '10133600', '10016900', '10146000', '10102250', '10150500', '10137500', '10172952', '10140100', '10129500', '10154200', '10153800', '10058600', '10146400', '10015700', '10164500', '10068500', '10156000', '10168500']


In [ ]:
type(streamflow_data) # To check the type of the streamflow_data variable itself


dict

In [ ]:
measurements.head()

,AquiferID,Well_ID,Date,WTE,State
0,1,381033113480701,2012-09-06,7092.99,UT
1,1,381037113474001,2012-09-06,7175.95,UT
2,1,381152113442801,1995-11-22,6200.00,UT
3,1,381236113485601,2014-07-23,7151.00,UT
4,1,382113113435401,2008-09-03,5395.95,UT


## 1.1 PCHIP interpolation of WTE and calculate Delta_WTE

In [ ]:
from scipy.interpolate import PchipInterpolator
import numpy as np

# Convert 'Date' column to datetime objects if not already done
measurements['Date'] = pd.to_datetime(measurements['Date'])

# Group data by 'Well'
grouped = measurements.groupby('Well_ID')

# Iterate through each well's data
interpolated_data = []
for well, data in grouped:
    if len(data) >= 2:
        # Create daily date range for the well's data period
        date_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='D')

        # Prepare x (date) and y (measurement) data for interpolation
        x = data['Date'].astype(np.int64) // 10**9  # Convert datetime to Unix timestamps in seconds
        y = data['WTE']

        # Create the PCHIP interpolator
        f = PchipInterpolator(x, y)

        # Interpolate for the full daily range
        x_new = date_range.astype(np.int64) // 10**9
        y_new = f(x_new)

        # Create a new DataFrame with daily interpolated data
        interpolated_df = pd.DataFrame({'Date': date_range, 'Well_ID': well, 'WTE': y_new})
        interpolated_data.append(interpolated_df)
    else:
        # Append original data if there are less than 2 data points
        interpolated_data.append(data)

# Concatenate all the interpolated dataframes
final_measurements = pd.concat(interpolated_data)

print(final_measurements.head())

   AquiferID          Well_ID       Date      WTE State
0        1.0  381033113480701 2012-09-06  7092.99    UT
1        1.0  381037113474001 2012-09-06  7175.95    UT
2        1.0  381152113442801 1995-11-22  6200.00    UT
3        1.0  381236113485601 2014-07-23  7151.00    UT
0        NaN  382113113435401 2008-09-03  5395.95   NaN


In [ ]:
if 'AquiferID' in final_measurements.columns:
    final_measurements = final_measurements.drop('AquiferID', axis=1)
if 'State' in final_measurements.columns:
    final_measurements = final_measurements.drop('State', axis=1)
final_measurements.head()

,Well_ID,Date,WTE
0,381033113480701,2012-09-06,7092.99
1,381037113474001,2012-09-06,7175.95
2,381152113442801,1995-11-22,6200.00
3,381236113485601,2014-07-23,7151.00
0,382113113435401,2008-09-03,5395.95


In [ ]:
# Group the final_measurements DataFrame by 'Well_ID'
grouped_wells = final_measurements.groupby('Well_ID')

# Store the results
delta_wte_data = []

# Iterate through each well
for well, data in grouped_wells:
    # Get the first WTE value for the well
    first_wte = data['WTE'].iloc[0]

    # Calculate Delta_WTE for each row in the well's data
    data['Delta_WTE'] = data['WTE'] - first_wte
    delta_wte_data.append(data)

# Concatenate the results back into a single DataFrame
final_measurements_delta = pd.concat(delta_wte_data)
final_measurements_delta.sort_values(by='Date', inplace=True)

print(final_measurements_delta.head())

                Well_ID       Date      WTE  Delta_WTE
172154  421030117205801 1906-01-01  4509.42        0.0
19619   400920112124701 1907-01-01  3915.00        0.0
160558  414705112281001 1910-01-01  4510.00        0.0
144834  412141112253701 1911-01-01  4225.00        0.0
147339  413335113543001 1911-01-01  4952.00        0.0


# 3.Create pairs

In [ ]:
def find_pairs(streamflow_data, final_measurements_delta, temp_file='paired_temp.csv'):
    """
    Finds pairs of well data and streamflow data based on matching dates and ML_BFD filter.
    Writes results to temp file to save memory.

    Args:
        streamflow_data (dict): Dictionary of streamflow DataFrames.
        final_measurements_delta (pd.DataFrame): DataFrame containing well data with Delta_WTE.
        temp_file (str): Temporary file to store results.

    Returns:
        pd.DataFrame: DataFrame containing the paired data with gage_id as the first column.
    """

    first_write = True  # Flag to check if it's the first write

    for gage_id, streamflow_df in streamflow_data.items():
        print(f"Working on Gage {gage_id}")
        start = time.time()

        # Filter streamflow data based on ML_BFD
        filtered_streamflow = streamflow_df[streamflow_df['ML_BFD'] == 1][['Date', 'Q']]
        print(f"Filtered Streamflow Data Shape: {filtered_streamflow.shape}")

        # Add gage_id to the filtered streamflow data
        filtered_streamflow['gage_id'] = gage_id

        # Perform the merge operation
        paired = filtered_streamflow.merge(
            merged_df[['Reach_ID', 'Reach_Elevation', 'Distance_to_Reach', 'Downstream_Gage', 'Well_ID', 'Date', 'Delta_WTE']],
            on='Date',
            how='inner'  # Only keep matches
        )

        # Reorder columns to place 'gage_id' first
        paired = paired[['gage_id'] + [col for col in paired.columns if col != 'gage_id']]

        # Write to file
        if not paired.empty:
            paired.to_csv(
                temp_file,
                mode='w' if first_write else 'a',  # 'w' for first write, 'a' for append
                header=first_write,  # Only write header for first file
                index=False
            )
            first_write = False

        # Clear memory
        del filtered_streamflow
        del paired

        print(f"Finished Gage {gage_id} with {(time.time()-start):.1f} seconds")

    # Read the complete file if any data was written
    if not first_write:  # If first_write is still True, no data was written
        return pd.read_csv(temp_file)
    else:
        return pd.DataFrame()  # Return empty DataFrame if no pairs found

In [ ]:
## find some samples

# Select a subset of gages for testing
selected_gages = list(streamflow_data.keys())[:5]  # Adjust the number as needed

# Create a temporary dictionary to hold the selected streamflow data
selected_streamflow_data = {}
for gage_id in selected_gages:
    selected_streamflow_data[gage_id] = streamflow_data[gage_id]

# Now use the selected_streamflow_data in the find_pairs function
paired_data = find_pairs(selected_streamflow_data, merged_df)


Working on Gage 10141000
Filtered Streamflow Data Shape: (825, 2)
Finished Gage 10141000 with 2.7 seconds
Working on Gage 10026500
Filtered Streamflow Data Shape: (3319, 2)
Finished Gage 10026500 with 20.2 seconds
Working on Gage 10041000
Filtered Streamflow Data Shape: (1533, 2)
Finished Gage 10041000 with 10.2 seconds
Working on Gage 10134500
Filtered Streamflow Data Shape: (11491, 2)
Finished Gage 10134500 with 47.9 seconds
Working on Gage 10157500
Filtered Streamflow Data Shape: (9100, 2)
Finished Gage 10157500 with 31.4 seconds


In [ ]:
paired_data

,gage_id,Date,Q,Reach_ID,Reach_Elevation,Distance_to_Reach,Downstream_Gage,Well_ID,Delta_WTE
0,10141000,2018-09-28,17.70,710871184.0,1474.0,127.374161,NaN,401312112442301,-2.988631
1,10141000,2018-09-28,17.70,710535816.0,1372.0,78.577347,10118000.0,413840111552601,5.775496
2,10141000,2018-09-28,17.70,710393017.0,1320.5,298.036185,NaN,404152111525101,0.664208
3,10141000,2018-09-28,17.70,710406851.0,1355.5,483.189098,NaN,414511112175401,-1.504321
4,10141000,2018-09-28,17.70,710571586.0,1581.0,103.679172,NaN,403447112173801,-31.198516
...,...,...,...,...,...,...,...,...,...
10356695,10157500,2023-12-17,0.24,710666822.0,1381.5,279.581154,10167000.0,402333111513401,-11.824432
10356696,10157500,2023-12-18,0.23,710666822.0,1381.5,279.581154,10167000.0,402333111513401,-11.756302
10356697,10157500,2023-12-19,0.22,710666822.0,1381.5,279.581154,10167000.0,402333111513401,-11.687854
10356698,10157500,2023-12-20,0.22,710666822.0,1381.5,279.581154,10167000.0,402333111513401,-11.619087


In [ ]:
test_df=find_pairs(streamflow_data, merge)

Working on Gage 10141000
Filtered Streamflow Data Shape: (825, 2)
Finished Gage 10141000 with 1.8 seconds
Working on Gage 10026500
Filtered Streamflow Data Shape: (3319, 2)
Finished Gage 10026500 with 17.7 seconds
Working on Gage 10041000
Filtered Streamflow Data Shape: (1533, 2)
Finished Gage 10041000 with 11.8 seconds
Working on Gage 10134500
Filtered Streamflow Data Shape: (11491, 2)
Finished Gage 10134500 with 47.9 seconds
Working on Gage 10157500
Filtered Streamflow Data Shape: (9100, 2)
Finished Gage 10157500 with 29.5 seconds
Working on Gage 10015900
Filtered Streamflow Data Shape: (1948, 2)
Finished Gage 10015900 with 13.5 seconds
Working on Gage 10092700
Filtered Streamflow Data Shape: (5811, 2)
Finished Gage 10092700 with 23.4 seconds
Working on Gage 10105900
Filtered Streamflow Data Shape: (8304, 2)
Finished Gage 10105900 with 30.7 seconds
Working on Gage 10132000
Filtered Streamflow Data Shape: (7076, 2)
Finished Gage 10132000 with 27.5 seconds
Working on Gage 10106000
Filt

KeyboardInterrupt: 